In [33]:
import numpy as np
import os
import neurokit2 as nk

class ECGAnalyzer:
    def __init__(self, data_dir, sr=500):
        self.data_dir = data_dir
        self.sr = sr
        self.data = self.load_npy_data(data_dir)
    
    def load_npy_data(self, data_dir):
        """
        주어진 디렉토리에서 모든 .npy 파일을 로드하여 numpy 배열로 반환하는 함수.
        """
        X = []
        for file_name in os.listdir(data_dir):
            if file_name.endswith('.npy'):
                data_path = os.path.join(data_dir, file_name)
                data = np.load(data_path).astype(np.float32)
                X.append(data)
        return np.array(X)
    
    def calculate_rr_intervals(self, r_peaks):
        """
        R-peak 위치와 샘플링 속도를 이용하여 RR 간격을 계산하는 함수.
        """
        return np.diff(r_peaks) / self.sr
    
    def find_pqrst_and_calculate_hr(self, signal):
        """
        ECG 신호에서 PQRST 피크를 찾고 심박수를 계산하는 함수.
        """
        try:
            signals = nk.ecg_clean(signal, self.sr, method='neurokit')
            _, rpeaks = nk.ecg_peaks(signals, sampling_rate=self.sr)
            _, waves_peak = nk.ecg_delineate(signals, rpeaks, sampling_rate=self.sr, method='peak')

            r_peaks = np.array(rpeaks['ECG_R_Peaks'])
            r_peaks = r_peaks[~np.isnan(r_peaks)].astype('int')

            p_peaks = np.array(waves_peak['ECG_P_Peaks'])
            q_peaks = np.array(waves_peak['ECG_Q_Peaks'])
            s_peaks = np.array(waves_peak['ECG_S_Peaks'])
            t_peaks = np.array(waves_peak['ECG_T_Peaks'])

            p_peaks = p_peaks[~np.isnan(p_peaks)].astype('int')
            q_peaks = q_peaks[~np.isnan(q_peaks)].astype('int')
            s_peaks = s_peaks[~np.isnan(s_peaks)].astype('int')
            t_peaks = t_peaks[~np.isnan(t_peaks)].astype('int')

            if len(r_peaks) < 2:
                return [], [], [], [], [], np.nan

            rr_intervals = self.calculate_rr_intervals(r_peaks)
            if len(rr_intervals) == 0 or np.mean(rr_intervals) == 0:
                return [], [], [], [], [], np.nan

            hr = 60 / np.mean(rr_intervals)
            return p_peaks, q_peaks, r_peaks, s_peaks, t_peaks, hr

        except Exception as e:
            print(f"Error: {e}")
            return [], [], [], [], [], np.nan
    
    def analyze_lead(self, sample_index, lead_index):
        """
        특정 샘플의 특정 리드를 분석하여 PQRST 피크와 심박수를 계산하는 함수.
        """
        signal = self.data[sample_index, lead_index]
        return self.find_pqrst_and_calculate_hr(signal)
    
    def analyze_all_leads(self, sample_index):
        """
        특정 샘플의 모든 리드를 분석하여 PQRST 피크와 심박수를 계산하는 함수.
        """
        results = {}
        for lead_index in range(12):
            results[f'Lead {lead_index + 1}'] = self.analyze_lead(sample_index, lead_index)
        return results

# 사용 예제
folder_path = "/data/hw/PTB-XL/PTB_XL_npy/"
analyzer = ECGAnalyzer(folder_path)

# 특정 샘플의 모든 리드를 분석
sample_index = 1
results = analyzer.analyze_all_leads(sample_index)

# 결과 출력
for lead, (p_peaks, q_peaks, r_peaks, s_peaks, t_peaks, hr) in results.items():
    print(f"{lead}:")
    print("P 피크:", p_peaks)
    print("Q 피크:", q_peaks)
    print("R 피크:", r_peaks)
    print("S 피크:", s_peaks)
    print("T 피크:", t_peaks)
    print("심박수 (HR):", hr)
    print("="*40)


Lead 1:
P 피크: [ 558 1032 1536 2018 2559 3112 3645 4171]
Q 피크: [ 592 1096 1590 2083 2607 3166 3709 4232]
R 피크: [ 638 1116 1612 2102 2631 3187 3728 4251 4770]
S 피크: [ 702 1146 1649 2144 2669 3226 3770 4296]
T 피크: [ 779 1253 1753 2244 2769 3329 3872 4391]
심박수 (HR): 58.08325266214907
Lead 2:
P 피크: [ 541 1032 1536 2017 2547 3647 4167]
Q 피크: [ 618 1097 1590 2082 2603 3696 4206]
R 피크: [ 638 1117 1612 2102 2630 3187 3728 4251 4770]
S 피크: [ 700 1150 1667 2148 2689 3226 3763 4292]
T 피크: [ 781 1258 1755 2245 2773 3332 3874 4394]
심박수 (HR): 58.083252662149086
Lead 3:
P 피크: [ 686 1087 1985 2510 2873 3134 3426 3628 4157]
Q 피크: [ 767 1099 1485 2042 2560 2892 3146 3477 3641 4205]
R 피크: [ 809 1117 1612 2102 2627 2904 3183 3489 3729 4251 4769]
S 피크: [ 835 1185 1731 2226 2638 2915 3197 3641 3855 4377]
T 피크: [ 849 1317 1779 2321 2649 3041 3227 3729 3888 4411]
심박수 (HR): 75.75757575757575
Lead 4:
P 피크: [ 524  996 1434 1987 2446 3579 4127]
Q 피크: [ 556 1032 1536 2017 2558 3112 3646 4224]
R 피크: [ 618 1097 1590 